In [ ]:
!pip install -q polyglot
!pip3 install -q pyicu
!pip3 install -q pycld2
!pip3 install -q morfessor
!pip install deep-translator
!wget https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/Project_CodeNet_metadata.tar.gz
!mkdir /content/ds

In [ ]:
from polyglot.detect import Detector
from deep_translator import GoogleTranslator


from IPython.display import HTML, display
from google.colab import output
import codecs
import glob
import random
import tarfile
import requests
import os
import shutil

from bs4 import BeautifulSoup
import re

import pandas as pd
import numpy as np
import pickle 

shutil.unpack_archive('/content/Project_CodeNet_metadata.tar.gz', '/content/ds')

In [ ]:
Detector(mixed_text).languages[0].code

'en'

In [ ]:
def findTask(text):
  start=text.find('\n\n')
  end=text.find('Input')
  filter=text[start+2:end-2].replace('\n\n', ' ').replace('\n', ' ').replace(' Problem Statement', '')
  if filter.find('Constraints')!=-1:
    filter=filter[:filter.find('Constraints')]
  return " ".join(re.sub(r'[^A-Za-z0-9 >+>=+<=+=+-]+', '', filter).split())

In [ ]:
def checkLanguage(text):
  lang=Detector(text).languages[0].code
  if lang=='en':
    return findTask(text)
  else:
    translated = GoogleTranslator(source='auto', target='en').translate(text)
    return findTask(translated)

In [ ]:
f=codecs.open("/content/ds/Project_CodeNet/problem_descriptions/p00005.html", 'r')
BeautifulSoup(f.read(), "lxml").text

'GCD and LCM\n\nWrite a program which computes the greatest common divisor (GCD) and the least common multiple (LCM) of given a and b. \n\nInput\n\nInput consists of several data sets. Each data set contains a and b separated by a single space in a line. The input terminates with EOF.\n\nConstraints\n\n 0 < a, b ≤ 2,000,000,000\n LCM(a, b) ≤ 2,000,000,000\n The number of data sets ≤ 50\n\nOutput\n\nFor each data set, print GCD and LCM separated by a single space in a line.\n\nSample Input\n\n8 6\n50000000 30000000\n\nOutput for the Sample Input\n\n2 24\n10000000 150000000\n\n'

In [ ]:
directory_path = '/content/ds/Project_CodeNet/problem_descriptions'
directory_files = os.listdir(directory_path)

problemNames=[]
for file in directory_files:
    try:
      f=codecs.open(os.path.join(directory_path, file), 'r')
      problemNames.append(checkLanguage(BeautifulSoup(f.read(), "lxml").text))
    except:
      problemNames.append('')

In [ ]:
with open('/content/drive/MyDrive/CodeNetProblems.txt', 'w') as f:
    for line in problemNames:
        f.write(f"{line}\n")

In [ ]:
problem_dict = dict(zip(directory_files, problemNames))

with open('/content/drive/MyDrive/problem_dict.pkl', 'wb') as f:
    pickle.dump(problem_dict, f)

In [ ]:
np.array(problemNames[0])

array('A railroad running from west to east in Atcoder Kingdom is now complete There are N stations on the railroad numbered 1 through N from west to east Tomorrow the opening ceremony of the railroad will take place On this railroad for each integer i such that 1iN-1 there will be trains that run from Station i to Station i+1 in Ci seconds No other trains will be operated The first train from Station i to Station i+1 will depart Station i Si seconds after the ceremony begins Thereafter there will be a train that departs Station i every Fi seconds Here it is guaranteed that Fi divides Si That is for each Time t satisfying Sit and tFi=0 there will be a train that departs Station i t seconds after the ceremony begins and arrives at Station i+1 t+Ci seconds after the ceremony begins where AB denotes A modulo B and there will be no other trains For each i find the earliest possible time we can reach Station N if we are at Station i when the ceremony begins ignoring the time needed to chang

In [ ]:
file_name = "Project_CodeNet_MLM.tar.gz"
data_url = f"https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/{file_name}"

if os.path.exists(file_name):
    os.remove(file_name) 
with open(file_name, "wb") as f:
    f.write(requests.get(data_url).content)
    
if os.path.exists("tokens"):
    shutil.rmtree("tokens")    
with tarfile.open(file_name) as f:
    f.extractall()

In [ ]:
directory_path = '/content/ds/Project_CodeNet/metadata'
directory_files = os.listdir(directory_path)
directory_files.remove('problem_list.csv')

df = pd.DataFrame()
for file in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, file), encoding = 'ISO-8859-1',low_memory=False)
    df = pd.concat([df, df_file])

In [ ]:
df[(df['status']=='Accepted') & ((df['original_language']=='C') | (df['language']=='C'))].to_csv('/content/drive/MyDrive/CodeNetSubmissions.csv', sep='\t', encoding='utf-8')
submissions=df[(df['status']=='Accepted') & ((df['original_language']=='C') | (df['language']=='C'))]
sub_dict=pd.Series(submissions.problem_id.values,index=submissions.submission_id).to_dict()
with open('/content/drive/MyDrive/sub_dict.pkl', 'wb') as f:
    pickle.dump(sub_dict, f)

In [ ]:
def get_text_list_from_files(files):
    text_list = []
    for name in files:
        with open(name) as f:
            for line in f:
                text_list.append(line)
    return text_list

def get_data_from_text_files(folder_name):
    files = glob.glob(folder_name + '/*.toks')
    code = get_text_list_from_files(files)
    temp=[sub_dict[x[len(f'{folder_name}/'):-len('.toks')]] for x in glob.glob(folder_name + '/*.toks')]
    nl_desc=[]
    for problem in temp:
      try:
        nl_desc.append(problem_dict[problem+'.html'])
      except:
        nl_desc.append('')
    df = pd.DataFrame({'nl': nl_desc, 'code': code})
    df = df.sample(len(df)).reset_index(drop=True)
    return df

train_data = get_data_from_text_files('tokens/train')
train_data

                                                  nl  \
0  Write a program that prints the surface area S...   
1  I did an exit poll on the amount of money spen...   
2  Write a program that prints the surface area S...   
3  Given are integers abc and d If x and y are in...   
4  Given are three integers A1 A2 and A3 If A1+A2...   

                                                code  
0  # include < id . id > # include < id . id > in...  
1  # include < id . id > int main ( void ) { unsi...  
2  # include < id . id > # include < id . id > in...  
3  # include < id . id > int main ( ) { long long...  
4  # include < id . id > int main ( ) { int id , ...  


In [ ]:
train_data.nl[1]

'I did an exit poll on the amount of money spent on purchases at a department store Create a program that takes data on shopping prices as input calculates the average shopping price per person and outputs the data The number of people surveyed shall be 100000 or less and the purchase amount per person shall not exceed 1 million yen'

In [ ]:
train_data.code[1]

'# include < id . id > int main ( void ) { unsigned long long int id , id , id ; unsigned long long int id = 0 ; double id ; scanf ( string , operator id ) ; for ( id = 0 ; id < id ; id operator ) { scanf ( string , operator id ) ; id operator id ; } id operator id ; printf ( string , id ) ; return 0 ; } '

In [ ]:
train_data[train_data.nl!=''].to_csv('/content/drive/MyDrive/CodeNetTrain.csv', sep='\t', encoding='utf-8')

In [ ]:
test_data = get_data_from_text_files('tokens/test')
test_data

In [ ]:
train_data[train_data.nl!=''].to_csv('/content/drive/MyDrive/CodeNetTrain.csv', sep='\t', encoding='utf-8')

In [ ]:
test_data = get_data_from_text_files('tokens/test')
test_data[test_data.nl!=''].to_csv('/content/drive/MyDrive/CodeNetTest.csv', sep='\t', encoding='utf-8')

,nl,code
0,An X-layered kagami mochi X 1 is a pile of X r...,# include < id . id > int main ( void ) { int ...
1,We have a deck consisting of N cards Each card...,# include < id . id > # include < id . id > in...
2,You are given a string S consisting of lowerca...,# include < id . id > # include < string . id ...
3,You are parking at a parking lot You can choos...,"# include < id . id > int main ( ) { int id , ..."
4,You have A 500-yen coins B 100-yen coins and C...,# include < id . id > # include < string . id ...
...,...,...
4995,We have a 33 square grid where each square con...,# include < id . id > int main ( void ) { char...
4996,You went shopping to buy cakes and donuts with...,# include < id . id > int main ( ) { int id ; ...
4997,We call a 4-digit integer with three or more c...,# include < id . id > char id [ number ] ; int...
4998,Square1001 has seen an electric bulletin board...,"# include < id . id > int main ( ) { int id , ..."


In [ ]:
X=train_data[train_data.nl!=''].nl.to_numpy()

In [ ]:
X=np.concatenate([train_data[train_data.nl!=''].nl.to_numpy(), test_data[test_data.nl!=''].nl.to_numpy()])
y=np.concatenate([train_data[train_data.nl!=''].code.to_numpy(), test_data[test_data.nl!=''].code.to_numpy()])

In [ ]:
with open('/content/drive/MyDrive/c_train/x.npy', 'rb') as f:
    x = np.load(f)
with open('/content/drive/MyDrive/c_train/y.npy', 'rb') as f:
    YearEnd = np.load(f)